<a href="https://colab.research.google.com/github/kamrulsaad/UITS_CSE_COURSES/blob/main/8th-semester/cse-426/projects/Project_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ID: 2125051041

Name: Kamrul Huda Sattari Saad

# Importing libraries and dataset

In [6]:
import networkx as nx
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [7]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

STOPWORDS = set(stopwords.words('english'))
print(STOPWORDS)

{'here', 'between', "aren't", "won't", 'i', 'needn', 'it', 'an', 'too', "you're", 'shan', 'not', 'they', "we're", 'herself', 'own', 'up', 'her', 'there', 'don', "wasn't", "couldn't", 'my', "isn't", 'more', 'our', "she'll", "he'll", 'other', 'over', 'such', 'by', "they're", 'why', "you'll", 'ma', 'his', 'only', 'again', 'aren', 'am', 'haven', 'few', 'myself', 'whom', "they'd", 'this', 'no', 'in', "hadn't", 'doing', "doesn't", 'm', 'do', 'under', 'very', 'yours', 'these', "it's", "it'd", "we'd", 'has', 'all', 'and', 'out', 'while', 'at', 'about', 'below', "shan't", 'before', 'themselves', 'some', "you'd", 'each', "i'm", 'mightn', "they'll", 'mustn', 'both', 'was', 'wouldn', 'then', 'a', 'ain', 'into', 'weren', 'on', 'just', "hasn't", 'ourselves', 'than', 'when', 'during', 'but', 'are', 'their', 'should', 'shouldn', 'them', "i've", 'or', 'most', 'because', 'after', 'from', 'nor', 'is', 's', 'that', 'those', 'hadn', 'your', 'can', "don't", 'we', 'isn', 'with', 'wasn', "haven't", "should've

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
from collections import defaultdict

# Inverted index: word -> set of URLs
inverted_index = defaultdict(set)
url_list = set()

In [9]:
# This dictionary will be used to build the connection between links
web_connection = {'source':[], 'target':[]}

In [10]:
import re
# This function will clean the content of web page in order to build the inverted index.
def clean_and_tokenize(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())  # Remove punctuation and lowercase
    tokens = text.split()
    return [t for t in tokens if t not in STOPWORDS and len(t) > 1]

In [11]:
from urllib.parse import urljoin, urlparse
# The crawl function has 5 parameters
# url = The url to crawl
# base_domain = the base domain of the url. During crawling, the crawler will ignore links from other domains

def crawl(url, base_domain, visited, visit_limit, limit):
    if limit==0 or len(visited)==visit_limit:
        return

    try:
        response = requests.get(url, timeout=5)
        if response.status_code != 200:
            return
    except requests.RequestException:
        return

    visited.add(url)
    print("-"*(10-limit), end=" ")
    print(f"Crawled: {url}")

    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.get_text(separator=' ', strip=True)
    words = clean_and_tokenize(text)

    for word in words:
        inverted_index[word].add(url)
        url_list.add(url)

    # Recursively follow links
    for tag in soup.find_all('a', href=True):
        link = urljoin(url, tag['href'])
        parsed = urlparse(link)

        # Store external links as connection
        web_connection['source'].append(url)
        web_connection['target'].append(link)

        if parsed.netloc == base_domain and link not in visited:
            crawl(link, base_domain, visited, visit_limit, limit-1)

In [12]:
def crawl_roots(root_urls, max_per_root=2, visit_limit=50):
    for root in root_urls:
        print(f"\nStarting crawl from: {root}")
        domain = urlparse(root).netloc
        visited = set()
        crawl(root, domain, visited, visit_limit, max_per_root)

In [13]:
seed_urls = [
    'https://www.edx.org',
    'https://www.udemy.com',
    'https://www.thinkific.com',
    'https://www.teachable.com',
    'https://www.learnworlds.com',
    'https://www.hippovideo.io',
    'https://www.dacast.com',
    'https://www.muvi.com',
    'https://www.proxlearn.com',
    'https://fullmind.com',
]

crawl_roots(seed_urls, max_per_root=2)


Starting crawl from: https://www.edx.org
-------- Crawled: https://www.edx.org
--------- Crawled: https://www.edx.org/
--------- Crawled: https://www.edx.org/learn/computer-science/harvard-university-cs50-s-introduction-to-computer-science
--------- Crawled: https://www.edx.org/executive-education/massachusetts-institute-of-technology-artificial-intelligence-implications-for-business-strategy
--------- Crawled: https://www.edx.org/masters/micromasters/mitx-supply-chain-management
--------- Crawled: https://www.edx.org/learn/government/harvard-university-american-government-constitutional-foundations
--------- Crawled: https://www.edx.org/learn/democracy/harvard-university-we-the-people-civic-engagement-in-a-constitutional-democracy
--------- Crawled: https://www.edx.org/search?q=data science
--------- Crawled: https://www.edx.org/search?q=ai
--------- Crawled: https://www.edx.org/search?q=finance
--------- Crawled: https://www.edx.org/search?q=business
--------- Crawled: https://www.e

/tmp/ipython-input-11-3075972795.py:21: XMLParsedAsHTMLWarning: It looks like you're using an HTML parser to parse an XML document.

Assuming this really is an XML document, what you're doing might work, but you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the Python package 'lxml' installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.

If you want or need to use an HTML parser on this document, you can make this warning go away by filtering it. To do that, run this code before calling the BeautifulSoup constructor:

    from bs4 import XMLParsedAsHTMLWarning
    import warnings

    warnings.filterwarnings("ignore", category=XMLParsedAsHTMLWarning)

  soup = BeautifulSoup(response.text, 'html.parser')


--------- Crawled: https://fullmind.com/comments/feed/
--------- Crawled: https://fullmind.com/wp-content/uploads/2011/09/Fulmind-experiences.png
--------- Crawled: https://fullmind.com/metodo-fullmind
--------- Crawled: https://fullmind.com/wp-login.php?action=register
--------- Crawled: https://fullmind.com/metodo-fullmind/istruzioni


In [14]:
# print inverted index
print("\nSample inverted index (first 20 words):")
for word in list(inverted_index.keys())[:20]:
    print(f"{word}: {list(inverted_index[word])}")


Sample inverted index (first 20 words):
edx: ['https://www.edx.org/learn/economics', 'https://www.edx.org/executive-education/massachusetts-institute-of-technology-artificial-intelligence-implications-for-business-strategy', 'https://www.edx.org/masters/masters-in-education-online', 'https://www.edx.org/bachelors/online-accounting-bachelor-degree', 'https://www.edx.org/', 'https://www.edx.org/masters/online-masters-in-data-science', 'https://www.edx.org/learn/python', 'https://www.edx.org/become', 'https://www.edx.org/learn/excel', 'https://www.edx.org/learn/software-engineering', 'https://www.edx.org/learn/artificial-intelligence', 'https://www.edx.org/learn/computer-programming', 'https://www.edx.org/bachelors/computer-data-sciences', 'https://www.edx.org/become/how-to-become-a-software-developer', 'https://www.edx.org/learn/architecture', 'https://www.edx.org/learn/government/harvard-university-american-government-constitutional-foundations', 'https://www.edx.org/masters/online-slp

In [15]:
# Print first 20 connections
for source, target in list(zip(web_connection['source'], web_connection['target']))[:20]:
    print(f"{source} -> {target}")

https://www.edx.org -> https://www.edx.org/
https://www.edx.org/ -> https://www.edx.org/
https://www.edx.org/ -> https://www.edx.org/learn/computer-science/harvard-university-cs50-s-introduction-to-computer-science
https://www.edx.org/ -> https://www.edx.org/executive-education/massachusetts-institute-of-technology-artificial-intelligence-implications-for-business-strategy
https://www.edx.org/ -> https://www.edx.org/masters/micromasters/mitx-supply-chain-management
https://www.edx.org/ -> https://www.edx.org/learn/government/harvard-university-american-government-constitutional-foundations
https://www.edx.org/ -> https://www.edx.org/learn/democracy/harvard-university-we-the-people-civic-engagement-in-a-constitutional-democracy
https://www.edx.org/ -> https://www.edx.org/search?q=data science
https://www.edx.org/ -> https://www.edx.org/search?q=ai
https://www.edx.org/ -> https://www.edx.org/search?q=finance
https://www.edx.org/ -> https://www.edx.org/search?q=business
https://www.edx.or

# Creating graph from web connections

In [16]:
web_graph = nx.DiGraph()
for source, target in zip(web_connection['source'], web_connection['target']):
  web_graph.add_edge(source, target) # Iterate through source and target lists using zip

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 30))

# Layout and draw
pos = nx.spring_layout(web_graph, k=0.5, seed=42)
nx.draw(
    web_graph,
    pos=pos,
    with_labels=True,
    node_size=500,
    node_color="skyblue",
    font_size=8,
    font_color="black",
    arrowsize=10
)
plt.tight_layout()
plt.show()

# Calculating PageRank Scores

In [ ]:
pagerank_scores = nx.pagerank(web_graph, alpha=0.85, max_iter=100, tol=1e-6)

# Search Function

In [ ]:
def search_engine(query, index, scores):
    query_terms = query.lower().split()
    results = set()
    for term in query_terms:
        if term in index:
            if not results:
                results = set(index[term])
            else:
                results = results.intersection(index[term])  # Find common websites

    # Sort results based on score
    ranked_results = []
    for website in results:
        if website in scores:
          ranked_results.append((website, scores[website]))
    ranked_results.sort(key=lambda x: x[1], reverse=True)

    return ranked_results

# Query and display results

In [ ]:
query = "math"
print(f"\nSearch Results for '{query}' using PageRank:")
results = search_engine(query, inverted_index, pagerank_scores)

for page, score in results:
    print(f"{page}: ({score})")